# [H2] Water – [03] Water blending & target profile optimizer

**Doel.** Dit notebook helpt u (i) waterprofielen **mengen** (blenden) en (ii) een mengverhouding zoeken die een **doelprofiel** zo goed mogelijk benadert.

**Wat u hier leert**
- (1) Blending als massabalans (mg/L blijft lineair bij mengen)
- (2) Afwegingen tussen ionen (wegingen/priority)
- (3) Een eenvoudige optimizer (zonder externe libraries)

> **Scope (belangrijk).** Dit is een onderwijsmodel: geen ionsterkte/activiteit, geen neerslag (CaCO₃), geen CO₂-evenwicht en geen zouttoevoegingen in deze oefening (die komen in een volgend notebook).  
> We optimaliseren enkel **mengverhoudingen** van bestaande waters.


## 1. Theorie (kort)

### 1.1 Lineaire menging (massabalans)
Voor ionconcentraties in mg/L geldt bij mengen (zonder reacties):

\[ c_{mix} = \sum_i x_i\,c_i \]

waarbij \(x_i\) de volumefractie is (\(x_i\ge 0\), \(\sum x_i = 1\)).

### 1.2 Doelprofiel & foutfunctie
We kiezen een doelprofiel \(c^*\) en minimaliseren een gewogen kwadratische fout:

\[ J(x)=\sum_k w_k\,(c_{mix,k}-c^*_k)^2 \]

De wegingsfactoren \(w_k\) drukken uit welke ionen belangrijker zijn voor uw toepassing.


## 2. Parameters

Vul hieronder uw beschikbare waters in (mg/L).  
Gebruik bijvoorbeeld:
- kraanwater (CW)
- omgekeerde osmose / demi (RO) ~ bijna 0
- tweede bron (BW) (bottled/mineraalwater, ander leidingwater, ...)

U kunt 2 of 3 waters gebruiken. De optimizer werkt voor **N waters**.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

IONS = ["Ca","Mg","Na","Cl","SO4","HCO3"]  # mg/L

# ===== Beschikbare waters (mg/L) =====
waters = {
    "CW":  {"Ca":35, "Mg":8,  "Na":15, "Cl":30, "SO4":40, "HCO3":120},
    "RO":  {"Ca":0,  "Mg":0,  "Na":0,  "Cl":0,  "SO4":0,  "HCO3":0},
    "BW":  {"Ca":80, "Mg":20, "Na":10, "Cl":15, "SO4":120,"HCO3":60},
}

# ===== Doelprofiel (mg/L) =====
target = {"Ca":60, "Mg":10, "Na":15, "Cl":60, "SO4":80, "HCO3":50}

# ===== Wegingen (belangrijkheid per ion) =====
# Groter = belangrijker in de optimalisatie.
weights = {"Ca":1, "Mg":1, "Na":0.5, "Cl":1, "SO4":1, "HCO3":1}

list(waters.keys())


## 3. Hulpfuncties
- Conversie dictionary → vector
- Mixprofiel berekenen
- Doelfout (cost) berekenen
- Simpele constrained optimizer (projected gradient)


In [ ]:
def dict_to_vec(d, ions=IONS):
    return np.array([float(d.get(k,0.0)) for k in ions], dtype=float)

def vec_to_dict(v, ions=IONS):
    return {k: float(v[i]) for i,k in enumerate(ions)}

def mix_profile(water_matrix, x):
    # water_matrix: shape (Nwaters, Nions)
    # x: shape (Nwaters,), sum=1, x>=0
    return x @ water_matrix  # linear mix

def cost(water_matrix, x, target_vec, w_vec):
    m = mix_profile(water_matrix, x)
    diff = (m - target_vec)
    return float(np.sum(w_vec * diff * diff))

def project_to_simplex(v):
    """Project v onto the probability simplex: x>=0, sum(x)=1."""
    v = np.asarray(v, dtype=float)
    n = v.size
    u = np.sort(v)[::-1]
    cssv = np.cumsum(u) - 1
    ind = np.arange(1, n+1)
    cond = u - cssv/ind > 0
    if not np.any(cond):
        return np.ones(n)/n
    rho = ind[cond][-1]
    theta = cssv[cond][-1]/rho
    w = np.maximum(v - theta, 0)
    s = w.sum()
    if s == 0:
        return np.ones(n)/n
    return w/s

def projected_gradient_optimize(water_matrix, target_vec, w_vec, x0=None,
                               lr=1e-3, steps=4000, verbose=False):
    """Minimize weighted least squares on simplex using projected gradient descent."""
    N = water_matrix.shape[0]
    if x0 is None:
        x = np.ones(N)/N
    else:
        x = project_to_simplex(x0)

    A = water_matrix  # N x M
    t = target_vec    # M

    history = []
    checkpoints = max(1, steps//200)

    for i in range(steps):
        mix = x @ A                  # (M,)
        diff = (mix - t)             # (M,)
        grad = 2.0 * (A @ (w_vec * diff))  # (N,)
        x = project_to_simplex(x - lr * grad)

        if i % checkpoints == 0:
            history.append(cost(A, x, t, w_vec))
            if verbose and i % max(1, steps//20) == 0:
                print(i, history[-1], x)

    return x, np.array(history, dtype=float)


## 4. Simulatie: optimaliseren van mengverhoudingen

We bouwen de matrix **A** (waters × ionen) en zoeken **x** op de simplex (x≥0, som=1).


In [ ]:
# Matrix opbouwen
water_names = list(waters.keys())
A = np.vstack([dict_to_vec(waters[name]) for name in water_names])  # N x M
t = dict_to_vec(target)
w = dict_to_vec(weights)

# Optimalisatie
x_opt, hist = projected_gradient_optimize(A, t, w, lr=2e-4, steps=8000)

mix_opt = mix_profile(A, x_opt)

print("=== Optimale volumefracties ===")
for name, xi in zip(water_names, x_opt):
    print(f"{name:>3s}: {xi*100:6.2f} %")

print("\n=== Resultaatprofiel (mg/L) ===")
for ion, val in zip(IONS, mix_opt):
    print(f"{ion:>5s}: {val:7.2f}   (doel {target[ion]:.1f})")

print(f"\nTotale gewogen fout J: {cost(A, x_opt, t, w):.3f}")


## 5. Visualisatie

- Balkplot: doel vs. mix
- Convergentieplot: kostfunctie doorheen iteraties


In [ ]:
# Plot 1: doel vs mix
x = np.arange(len(IONS))
plt.figure(figsize=(9,4.8))
plt.bar(x-0.2, t, width=0.38, label="Doel")
plt.bar(x+0.2, mix_opt, width=0.38, label="Mix (opt)")
plt.xticks(x, IONS)
plt.ylabel("mg/L (ppm)")
plt.title("Doelprofiel vs. optimale blend")
plt.grid(True, axis="y", alpha=0.25)
plt.legend()
plt.tight_layout()
plt.show()

# Plot 2: convergence
plt.figure(figsize=(7.5,4.2))
plt.plot(np.arange(len(hist)), hist)
plt.xlabel("Checkpoints (subsamples van iteraties)")
plt.ylabel("Gewogen fout J")
plt.title("Convergentie (projected gradient)")
plt.grid(True, alpha=0.25)
plt.tight_layout()
plt.show()


## 6. Interpretatievragen

1. Zet **RO** uit (verwijder RO uit `waters`). Hoe verandert de optimale mix? Welke ionen worden moeilijker te matchen?  
2. Maak **Cl** dubbel zo belangrijk (`weights["Cl"]=2`). Wat is het effect op de mix en op SO₄?  
3. Kies een “hop-forward” doel (bv. hoger SO₄, lager Cl). Welke blend-resultaten krijgt u?  
4. Welke beperkingen ziet u in dit model t.o.v. echte waterchemie (neerslag, alkaliniteit vs. HCO₃⁻, pH-effecten)?  
5. Breid uit: voeg een constraint toe “RO ≤ 40%” en implementeer dit door na projectie `x[RO] = min(x[RO], 0.40)` te doen en opnieuw te normaliseren.


## 7. Uitbreidingen (optioneel)

- Voeg een **maximum** voor Na in (smaak/gezondheid) en optimaliseer met een penalty-term.  
- Voeg een volgende stap toe: **zouttoevoegingen** bovenop de blend (koppeling met notebook 2).  
- Bouw een “style presets” dictionary met typische doelprofielen (pils, IPA, stout, ...).  
